# Case Study - The Current

* The Current is an alternative radio station
* We will pull information about the play list.

## <font color="red">Exercise - Go the the following page and inspect the following </font>

* Song title
* Artist
* Play time
* Day, date, period (am/pm)

http://www.thecurrent.org/playlist/2014-01-01/01

In [1]:
import requests
import bs4
import datetime

In [2]:
example_url = 'http://www.thecurrent.org/playlist/2014-01-01/01'
s = requests.Session()
r = s.get(example_url)

soup = bs4.BeautifulSoup(r.content, "lxml")

# Pull off the period of the day (am/pm)

Pull out the "am"/"pm"

In [3]:
period = soup('span', class_="hour-header open")[0].next.split('to')[0].rstrip()[-2:]
period

'am'

## Day of the Week

* Pull out the day of the week

In [4]:
day_of_week = soup('a', class_="start-picker")[0].next.split(',')[0]
day_of_week

'Wednesday'

# Title of each song



In [5]:
titles = [tag.next.strip() for tag in soup.findAll('h5', class_ = "title")]
titles

['Holy Roller',
 'Kingdom of Rust',
 'Black Dog',
 'Turn It Around',
 'Flavor of the Month',
 'Potential Wife',
 '24 Hours',
 "Who's Gonna Shoe Your Pretty Little Feet?",
 'Marigold',
 'High Road',
 'The Vampyre Of Time and Memory',
 'Valerie Plame',
 'Morning Song',
 '(You Will) Set The World On Fire',
 'Sixteen Saltines',
 'Wave of Mutilation']

##  Name of the Artist

In [6]:
artists = [tag.next.strip() for tag in  soup.findAll('h5',class_='artist')]
artists

['Thao and The Get Down Stay Down',
 'Doves',
 'Frankie Lee',
 'Lucius',
 'The Posies',
 'Strange Names',
 'Sky Ferreira',
 'Billie Joe and Norah',
 'J. Roddy Walston and The Business',
 'Cults',
 'Queens of the Stone Age',
 'The Decemberists',
 'The Avett Brothers',
 'David Bowie',
 'Jack White',
 'Pixies']

##  Song Start Time 

In [7]:
start_times = [tag.time.next.strip() for tag in  soup('div', class_="two columns songTime")]
start_times

['1:59',
 '1:54',
 '1:51',
 '1:46',
 '1:44',
 '1:38',
 '1:34',
 '1:31',
 '1:27',
 '1:23',
 '1:19',
 '1:13',
 '1:09',
 '1:05',
 '1:03',
 '1:01']

# Putting it all together

In [9]:
def get_period(soup):
    search = soup('span', class_="hour-header open")
    if len(search) > 0:
        return search[0].next.split('to')[0].rstrip()[-2:]
    else:
        return None

def get_day(soup):
    search = soup('a', class_="start-picker")
    if len(search) > 0:
        return search[0].next.split(',')[0]
    else:
        return None
    
def get_song_info(url):
    print("Starting {0} urls".format(url))
    date = url.split('/')[-2]
    s = requests.Session()
    r = s.get(url)
    soup = bs4.BeautifulSoup(r.content, 'lxml')
    period = get_period(soup)
    day_of_week = get_day(soup)
    soup = bs4.BeautifulSoup(r.content)
    titles = [t.next.strip() for t in soup.findAll('h5', class_="title")]
    artists = [a.next.strip() for a in soup.findAll('h5',class_='artist')]
    times = [d.time.next.strip() for d in soup('div', class_="two columns songTime")]
    song_info = [(day_of_week, date, time, period, title, artist) 
             for time, title, artist in zip(times, titles, artists)]
    return song_info

In [10]:
get_song_info(example_url)

Starting http://www.thecurrent.org/playlist/2014-01-01/01 urls


/Users/tiverson/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/tiverson/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


[('Wednesday',
  '2014-01-01',
  '1:59',
  'am',
  'Holy Roller',
  'Thao and The Get Down Stay Down'),
 ('Wednesday', '2014-01-01', '1:54', 'am', 'Kingdom of Rust', 'Doves'),
 ('Wednesday', '2014-01-01', '1:51', 'am', 'Black Dog', 'Frankie Lee'),
 ('Wednesday', '2014-01-01', '1:46', 'am', 'Turn It Around', 'Lucius'),
 ('Wednesday',
  '2014-01-01',
  '1:44',
  'am',
  'Flavor of the Month',
  'The Posies'),
 ('Wednesday', '2014-01-01', '1:38', 'am', 'Potential Wife', 'Strange Names'),
 ('Wednesday', '2014-01-01', '1:34', 'am', '24 Hours', 'Sky Ferreira'),
 ('Wednesday',
  '2014-01-01',
  '1:31',
  'am',
  "Who's Gonna Shoe Your Pretty Little Feet?",
  'Billie Joe and Norah'),
 ('Wednesday',
  '2014-01-01',
  '1:27',
  'am',
  'Marigold',
  'J. Roddy Walston and The Business'),
 ('Wednesday', '2014-01-01', '1:23', 'am', 'High Road', 'Cults'),
 ('Wednesday',
  '2014-01-01',
  '1:19',
  'am',
  'The Vampyre Of Time and Memory',
  'Queens of the Stone Age'),
 ('Wednesday',
  '2014-01-01',


# Collecting a years worth of data

## Step 1 - Identify the url pattern

The Current uses urls of the following pattern

    'http://www.thecurrent.org/playlist/2017-05-04/10'

or 

    'http://www.thecurrent.org/playlist/year-month-day/hour'

## Question: How might you generate all combinations for a given year?

**Answer:** Python has a tool for that!

In [102]:
numdays = 365
base = datetime.datetime.today()
dts = [base - datetime.timedelta(hours = h) for h in range(0, numdays*24)]

In [103]:
def output_address(dt):
    fmt = 'http://www.thecurrent.org/playlist/%Y-%m-%d/%H'
    return dt.strftime(fmt)

def test_output_address():
    date = datetime.datetime(2000,1,1,1)
    assert output_address(date) == 'http://www.thecurrent.org/playlist/2000-01-01/01'
test_output_address()

In [104]:
urls = [output_address(d) for d in dts]
urls[:10]

['http://www.thecurrent.org/playlist/2017-05-14/16',
 'http://www.thecurrent.org/playlist/2017-05-14/15',
 'http://www.thecurrent.org/playlist/2017-05-14/14',
 'http://www.thecurrent.org/playlist/2017-05-14/13',
 'http://www.thecurrent.org/playlist/2017-05-14/12',
 'http://www.thecurrent.org/playlist/2017-05-14/11',
 'http://www.thecurrent.org/playlist/2017-05-14/10',
 'http://www.thecurrent.org/playlist/2017-05-14/09',
 'http://www.thecurrent.org/playlist/2017-05-14/08',
 'http://www.thecurrent.org/playlist/2017-05-14/07']

In [ ]:
from toolz import pipe
from toolz.curried import take
from more_itertools import side_effect, consume

with open('the_current_last_year.csv', 'w') as outfile:
    header = "Weekday,Date,Time,Period,Song_Title,Artist"
    print(header, file=outfile)
    count = 0
    for url in urls:
        for song_info in get_song_info(url):
            count += 1
            if count % 5 == 0:
                print("Processed {0} songs".format(count))
            join_info = ",".join(song_info)
            print(join_info, file=outfile)